<!-- TABS -->
# Retrieval augmented generation

In [ ]:
datas = [{'x': d} for d in data]

## Create vector-index

In [ ]:
from superduperdb import VectorIndex, Listener

vector_index_name = 'vector-index'

jobs, _ = db.add(
    VectorIndex(
        vector_index_name,
        indexing_listener=Listener(
            key=indexing_key,      # the `Document` key `model` should ingest to create embedding
            select=select,       # a `Select` query telling which data to search over
            model=embedding_model,         # a `_Predictor` how to convert data to embeddings
            uuid="embedding"
        )
    )
)
query_table_or_collection = select.table_or_collection

In [ ]:
query = "Tell me about the SuperDuperDB"

## Answer question with LLM

In [ ]:
from superduperdb import model
from superduperdb.components.graph import Graph, input_node

prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "{context}\n\n"
    "Here's the question: {query}"
)


@model
def build_prompt(query, docs):
    chunks = [doc["text"] for doc in docs]
    context = "\n\n".join(chunks)
    prompt = prompt_template.format(context=context, query=query)
    return prompt
    

# We build a graph to handle the entire pipeline

# create a input node, only have one input parameter `query`
in_ = input_node('query')
# pass the query to the vector search model
vector_search_results = vector_search_model(query=in_)
# pass the query and the search results to the prompt builder
prompt = build_prompt(query=in_, docs=vector_search_results)
# pass the prompt to the llm model
answer = llm(prompt)
# create a graph, and the graph output is the answer
rag = answer.to_graph("rag")
print(rag.predict(query)[0])

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [ ]:
db.add(rag)

You can now load the model elsewhere and make predictions using the following command.

```python
rag = db.load("model", 'context_llm')
print(rag.predict("Tell me about the SuperDuperDB")[0])
```